In [1]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [2]:
TRAIN_TEXT_FILE_PATH = './data/dataset.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()

text_sample = ' '.join(text_sample)

In [3]:
def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])

    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'и', 'н', 'т', 'с', 'л', 'р', 'в', 'м', 'к', 'д', 'у', ',', 'п', 'я', 'ы', 'г', 'б', 'з', 'ь', 'ч', 'й', '.', 'ж', 'х', 'ш', 'ю', 'ц', 'щ', 'э', 'С', '–', '-', '«', '»', 'ф', 'П', 'Н', 'М', 'Д', '\n', 'К', 'В', 'О', 'Б', 'Я', 'А', 'Р', 'Г', '1', '9', '(', ')', 'И', 'Т', 'Л', 'Ф', 'Э', ':', ';', 'Ш', '…', '8', 'Е', 'У', 'e', 'n', 'Ч', 'ъ', '4', 'З', 'Х', '?', 'M', 't', 'o', 'r', 'B', '2', '3', '!', 'Й', '7', 'a', 's', 'Ю', 'T', 'h', 'i', 'P', 'u', '0', 'G', 'l', 'g', 'k', 'm', 'I', '5', '6', 'ё', 'C', 'R', '’', 'A', 'c', 'y', 'N', 'w', 'Y', 'Ж', 'O', 'd', 'v', 'b']


In [4]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [5]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char

    return predicted_text

In [6]:
class PozRNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(PozRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = PozRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 2000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

In [8]:
model.eval()

print(evaluate(
    model,
    char_to_idx,
    idx_to_char,
    temp=0.3,
    prediction_len=1000,
    start_text=' '
    )
)

 разовить команд – «Нью-йоркские Янки». Поэтому Бобби, и забыл думать в еврейской деятельности, его вышла в 1980 году, когда мы особенно себе, что в стране понятию стороне?»); затем их удивление первых пяти лет спед деле вызывало то не говорить не буду. Понятно, которые первой русский, а я прессиольно в черных следило задача по советские женщины, которые первыми в истории бейсбола стать некогда в 1982 году был выпущен спичечный которой стал пока не менее сообразительных полнейший советских годол, ресилен в естерстве потом самой советского домой каждый и делеком во время игр на легендарном «Янки-стадионе». Господи, что я потом, который должен был открываться пришлось в одиночку добывать мирны на всеобщее особенно себе, что в главным из всех бейсбольных команд – «Нью-йоркские Янки». Поэтому Бобби, я начав ее назила еврейской команд играли три мысли по коломительной команд – «Нью-йоркские Янки». Поэтому Бобби, и забыл думать об этом смысле «Доджерс» ничем течение в головавшей хорошо за тр